In [1]:
import datetime
import numpy as np
import pandas as pd
import yfinance as yf

## Question 1

In [2]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
}

url = "https://stockanalysis.com/ipos/filings/"

def price_range_converter(s: str):
    if s == '-':
        return np.nan
    s = s.replace("$", "")
    return tuple(float(x) for x in s.split("-"))

dt_convert = lambda s: datetime.datetime.strptime(s, "%b %d, %Y")
float_convert = lambda s: np.nan if s == '-' else float(s)
price_convert = lambda s: np.nan if s  == '-' else float(s.replace("$", ""))
perc_convert = lambda s: np.nan if s == '-' else float(s.replace("%", "")) * 1e-2

convert = {
    "Filing Date": dt_convert,
    "Shares Offered": float_convert,
    "Price Range": price_range_converter,
    "IPO Date": dt_convert,
    "IPO Price": price_convert,
    "Current": price_convert,
    "Return": perc_convert,
}

ipos = pd.read_html(url, converters=convert, storage_options=headers)[0]
ipos.set_index("Filing Date", inplace=True)
ipos["Avg_price"] = ipos["Price Range"].apply(lambda x: np.mean(x))
ipos["Shares_offered_value"] = ipos["Shares Offered"] * ipos["Avg_price"] 

ipos.info()
ipos.head()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 329 entries, 2024-04-26 to 2018-12-27
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Symbol                329 non-null    object 
 1   Company Name          329 non-null    object 
 2   Price Range           258 non-null    object 
 3   Shares Offered        253 non-null    float64
 4   Avg_price             258 non-null    float64
 5   Shares_offered_value  249 non-null    float64
dtypes: float64(3), object(3)
memory usage: 18.0+ KB


,Symbol,Company Name,Price Range,Shares Offered,Avg_price,Shares_offered_value
Filing Date,,,,,,
2024-04-26,EURK,Eureka Acquisition Corp,"(10.0,)",5000000.0,10.0,50000000.0
2024-04-26,HDL,Super Hi International Holding Ltd.,NaN,NaN,NaN,NaN
2024-04-22,DRJT,Derun Group Inc,"(5.0,)",NaN,5.0,NaN
2024-04-19,GPAT,GP-Act III Acquisition Corp.,"(10.0,)",25000000.0,10.0,250000000.0
2024-04-16,JLJT,Jialiang Holdings Ltd,"(5.0,)",NaN,5.0,NaN


In [3]:
mask = (ipos.index.dayofweek == 4) & (ipos.index.year == 2023)
assert mask.sum() == 32
assert (~ipos["Shares_offered_value"][mask].isnull()).sum() == 24
ipos.loc[mask, "Shares_offered_value"].sum()  * 1e-6

275.95

## Question 2

In [4]:
ipo_2023 = pd.read_html("https://stockanalysis.com/ipos/2023/", converters=convert, storage_options=headers)[0]
ipo_2024 = pd.read_html("https://stockanalysis.com/ipos/2024/", converters=convert, storage_options=headers)[0]
ipo_34 = pd.concat([ipo_2023, ipo_2024])
ipo_34

,IPO Date,Symbol,Company Name,IPO Price,Current,Return
0,2023-12-27,IROH,Iron Horse Acquisitions Corp.,10.00,10.03,0.0030
1,2023-12-19,LGCB,Linkage Global Inc,4.00,3.24,-0.1900
2,2023-12-15,ZKH,ZKH Group Limited,15.50,12.43,-0.1981
3,2023-12-15,BAYA,Bayview Acquisition Corp,10.00,10.18,0.0180
4,2023-12-14,INHD,Inno Holdings Inc.,4.00,0.63,-0.8424
...,...,...,...,...,...,...
58,2024-01-18,PSBD,Palmer Square Capital BDC Inc.,16.45,16.43,-0.0012
59,2024-01-18,CCTG,CCSC Technology International Holdings Limited,6.00,2.50,-0.5833
60,2024-01-12,SYNX,Silynxcom Ltd.,4.00,3.24,-0.1900
61,2024-01-11,SDHC,Smith Douglas Homes Corp.,21.00,27.25,0.2976


In [5]:
ipo_tickers = ipo_34[ipo_34["IPO Date"] < datetime.datetime(2024, 3, 1)]["Symbol"]
assert len(ipo_tickers) == 185

ipo_tickers

0     IROH
1     LGCB
2      ZKH
3     BAYA
4     INHD
      ... 
58    PSBD
59    CCTG
60    SYNX
61    SDHC
62    ROMA
Name: Symbol, Length: 185, dtype: object

In [6]:
ticker_conv = {"IBAC": "IBACU", "RYZB": None, "PTHR": "PTHRU"}
ipo_growth = np.full((len(ipo_tickers), 30), np.nan)

for i, symbol in enumerate(ipo_tickers):
    
    ticker_name = ticker_conv.get(symbol, symbol)
    if ticker_name is None:
        continue
    history = yf.download(ticker_name, period="max", interval="1d", progress=False)
    adj_closes = history["Adj Close"]
    ipo_growth[i, :len(adj_closes)-1] = adj_closes.iloc[1:31] / adj_closes.iloc[0]

print(ipo_growth)

[[1.00099902 1.00049951 1.00099902 ... 0.99700293 0.99800195 0.99800195]
 [0.81122449 0.87755102 0.79591832 ... 0.79591832 0.89285713 0.99489796]
 [1.         1.         1.01161292 ... 1.14516129 1.16903231 1.10064513]
 ...
 [1.01108039 1.03047095 0.98060944 ... 0.98393356 0.96952911 0.98947372]
 [0.99130439 1.01076606 1.02898554 ... 1.19378884 1.16356111 1.17184264]
 [1.36206894 1.39310339 1.3896552  ... 0.4827586  0.52758618 0.63103448]]


In [7]:
mask = np.all(np.isfinite(ipo_growth), 1)
np.quantile(ipo_growth[mask, :], .75, axis=0).argmax() + 1

28

In [8]:
ipo_growth[:, 27]

array([0.99700293, 0.79591832, 1.14516129, 1.011988  , 0.0778659 ,
       1.00295273, 0.78137254, 1.08558561, 1.01190475, 0.53970588,
       1.1333333 , 0.88291744, 0.94687498, 1.00895524, 0.44559587,
       0.97199999, 1.64942139, 1.00695141, 1.00769231, 0.41860468,
       1.14427859, 0.95111116, 3.23409073, 0.97546342, 1.11797746,
       1.95652178, 1.040796  , 0.21600001, 2.73600006, 0.92500001,
       0.2470852 , 1.00993052, 1.00496034, 1.12053571, 0.57393335,
       0.50333333, 0.226     , 0.90322582, 0.24950099, 0.83440002,
       0.84489913, 0.73916912, 0.62376237,        nan, 0.63470588,
       0.82355714, 1.02040814, 1.08833329, 0.38651315, 0.3505535 ,
       1.        , 0.40963854, 0.04694512, 0.17358974, 0.65333335,
       1.06611564, 0.68691586, 1.01250005, 0.75332347, 0.32028986,
       0.6392157 , 0.36111111, 1.00196652, 1.00887566, 0.34162303,
       0.40000003, 0.20930231, 1.02727274, 0.90321905, 1.15638247,
       1.00788954, 0.71912228, 1.01084809, 0.32478633, 0.37349

In [9]:
np.quantile(ipo_growth, .75, axis = 0)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan])

This strategy looks bad, even at the optimal number of days you may lose all of the investment, and the top 75% gain only around +2%.

## Question 3

In [10]:
US_STOCKS = ['MSFT', 'AAPL', 'GOOG', 'NVDA', 'AMZN', 'META', 'BRK-B', 'LLY', 'AVGO','V', 'JPM']
EU_STOCKS = ['NVO','MC.PA', 'ASML', 'RMS.PA', 'OR.PA', 'SAP', 'ACN', 'TTE', 'SIE.DE','IDEXY','CDI.PA']
INDIA_STOCKS = ['RELIANCE.NS','TCS.NS','HDB','BHARTIARTL.NS','IBN','SBIN.NS','LICI.NS','INFY','ITC.NS','HINDUNILVR.NS','LT.NS']
LARGEST_STOCKS = US_STOCKS + EU_STOCKS + INDIA_STOCKS 
NEW_US = ['TSLA','WMT','XOM','UNH','MA','PG','JNJ','MRK','HD','COST','ORCL']
NEW_EU = ['PRX.AS','CDI.PA','AIR.PA','SU.PA','ETN','SNY','BUD','DTE.DE','ALV.DE','MDT','AI.PA','EL.PA']
NEW_INDIA = ['BAJFINANCE.NS','MARUTI.NS','HCLTECH.NS','TATAMOTORS.NS','SUNPHARMA.NS','ONGC.NS','ADANIENT.NS','ADANIENT.NS','NTPC.NS','KOTAKBANK.NS','TITAN.NS']
LARGE_STOCKS = NEW_EU + NEW_US + NEW_INDIA
start_date = "2013-01-01"
end_date = "2023-12-31"

In [11]:
largest_history = yf.download(LARGEST_STOCKS, period="max", interval="1d", start=start_date, end=end_date)
largest_history

[*********************100%%**********************]  33 of 33 completed


Price        Adj Close                                                   \
Ticker            AAPL         ACN        AMZN        ASML         AVGO   
Date                                                                      
2013-01-01         NaN         NaN         NaN         NaN          NaN   
2013-01-02   16.747723   56.201344   12.865500   59.563492    24.111876   
2013-01-03   16.536331   55.997902   12.924000   58.314785    24.237841   
2013-01-04   16.075716   56.307152   12.957500   57.717178    24.082230   
2013-01-07   15.981153   56.062996   13.423000   56.780643    23.948849   
...                ...         ...         ...         ...          ...   
2023-12-22  193.353287  351.798248  153.419998  749.634766  1117.222046   
2023-12-26  192.803986  350.785828  153.410004  759.745667  1127.089966   
2023-12-27  192.903839  351.172943  153.339996  761.090454  1121.394287   
2023-12-28  193.333298  348.959625  153.380005  754.934204  1117.650269   
2023-12-29  192.284637  348.284729  151.940002  754.007751  1111.516357   

Price                                                                    ...  \
Ticker     BHARTIARTL.NS       BRK-B      CDI.PA        GOOG        HDB  ...   
Date                                                                     ...   
2013-01-01    273.840576         NaN         NaN         NaN        NaN  ...   
2013-01-02    272.947906   93.199997  103.484512   18.013729  19.166071  ...   
2013-01-03    278.856628   93.620003  104.575058   18.024191  19.031134  ...   
2013-01-04    278.006470   93.849998  104.107681   18.380356  19.128849  ...   
2013-01-07    277.836456   93.449997  103.796089   18.300158  18.965988  ...   
...                  ...         ...         ...         ...        ...  ...   
2023-12-22    989.549988  356.470001  700.377502  142.720001  67.000000  ...   
2023-12-26    999.849976  356.829987         NaN  142.820007  66.720001  ...   
2023-12-27   1021.200012  356.950012  695.922852  141.440002  67.040001  ...   
2023-12-28   1036.699951  357.570007  697.902710  141.279999  67.220001  ...   
2023-12-29   1032.199951  356.660004  700.377502  140.929993  67.110001  ...   

Price          Volume                                                        \
Ticker            NVO     OR.PA RELIANCE.NS   RMS.PA        SAP     SBIN.NS   
Date                                                                          
2013-01-01        NaN       NaN   3415675.0      NaN        NaN  17202300.0   
2013-01-02  1380000.0  655291.0   6720949.0  16407.0   686500.0  14660120.0   
2013-01-03  2462000.0  464918.0   8633404.0  23190.0   990000.0  20103090.0   
2013-01-04  1611000.0  408067.0   6653188.0  44563.0  1485400.0  13705950.0   
2013-01-07  2802000.0  504076.0   7653590.0  65288.0  1193300.0  10264490.0   
...               ...       ...         ...      ...        ...         ...   
2023-12-22  2211100.0  172262.0   8270892.0  57612.0   448400.0  14998068.0   
2023-12-26  1851400.0       NaN   3732832.0      NaN   279700.0  10153089.0   
2023-12-27  2254900.0  128642.0   4602078.0  39591.0   911600.0  14417646.0   
2023-12-28  2380100.0  165203.0   6151318.0  35291.0  1295500.0  16982092.0   
2023-12-29  2055900.0  137134.0   5432292.0  24471.0   432200.0  13221898.0   

Price                                                    
Ticker         SIE.DE     TCS.NS        TTE           V  
Date                                                     
2013-01-01        NaN   509580.0        NaN         NaN  
2013-01-02  3366503.0  1880246.0  1178200.0  15826400.0  
2013-01-03  1764027.0  2635130.0  1197000.0  14937200.0  
2013-01-04  1723418.0  5228760.0   881600.0  10376000.0  
2013-01-07  1715844.0  1820752.0  1306600.0  10242400.0  
...               ...        ...        ...         ...  
2023-12-22   981828.0  2413058.0   878700.0   5111500.0  
2023-12-26        NaN  1285231.0   504700.0   2085100.0  
2023-12-27   752828.0  1293976.0   934400.0   4034700.0  
2023-12-28   633466.0  16

In [12]:
growth_7d_largest = largest_history.loc[:, "Adj Close"] / largest_history.loc[:, 'Adj Close'].shift(7)
largest_mean = growth_7d_largest.mean(1)

In [13]:
large_history = yf.download(LARGE_STOCKS, period="max", interval="1d", start=start_date, end=end_date)
growth_7d_large = large_history.loc[:, "Adj Close"] / large_history.loc[:, 'Adj Close'].shift(7)
large_mean = growth_7d_large.mean(1)

[*********************100%%**********************]  33 of 33 completed


In [14]:
large_largest_comp = pd.concat([largest_mean, large_mean], axis=1, keys = ["largest", "large"])
large_largest_comp.loc[large_largest_comp.index >= "2014-01-01"]

,largest,large
Date,,
2014-01-01,1.011797,1.009288
2014-01-02,0.998189,0.999348
2014-01-03,0.996692,0.999633
2014-01-06,0.990353,0.990707
2014-01-07,0.987984,0.990938
...,...,...
2023-12-22,1.013788,1.003809
2023-12-26,1.014900,1.014441
2023-12-27,1.013800,1.003292


In [15]:
(large_largest_comp["large"] > large_largest_comp["largest"]).mean()

0.4726699369306237

## Question 4

In [132]:
stocks_prices = yf.download(LARGEST_STOCKS, period="max", interval="1d", start="2013-01-01", end="2024-01-31").stack(future_stack=True).dropna()

[*********************100%%**********************]  33 of 33 completed


In [133]:
def cci(df: pd.DataFrame, ndays: int) -> pd.Series:
    typical_price: pd.Series = (df["High"] + df["Low"] + df["Close"]) / 3
    moving_average = typical_price.rolling(ndays).mean()
    mean_deviation = typical_price.rolling(ndays).apply(lambda x: (x - x.mean()).abs().mean())
    cci = (typical_price - moving_average) / (0.015 * mean_deviation)
    return cci

In [134]:
stocks_prices = stocks_prices.reorder_levels(["Ticker", "Date"]).sort_index()

In [139]:
stocks_prices["cci"] = stocks_prices.groupby(level="Ticker", group_keys=False).apply(cci, ndays=14)
stocks_prices

Price               Adj Close       Close        High         Low        Open  \
Ticker Date                                                                     
AAPL   2013-01-02   16.747723   19.608213   19.821428   19.343929   19.779285   
       2013-01-03   16.536331   19.360714   19.631071   19.321428   19.567142   
       2013-01-04   16.075722   18.821428   19.236786   18.779642   19.177500   
       2013-01-07   15.981154   18.710714   18.903570   18.400000   18.642857   
       2013-01-08   16.024157   18.761070   18.996071   18.616072   18.900356   
...                       ...         ...         ...         ...         ...   
V      2024-01-24  271.144409  271.649994  272.859985  271.079987  272.570007   
       2024-01-25  272.102600  272.609985  272.779999  270.799988  271.950012   
       2024-01-26  267.441315  267.940002  270.869995  265.600006  265.940002   
       2024-01-29  273.150665  273.660004  275.070007  266.709991  266.970001   
       2024-01-30  276.634155  277.149994  278.640015  274.190002  274.589996   

Price                   Volume         cci  
Ticker Date                                 
AAPL   2013-01-02  560518000.0         NaN  
       2013-01-03  352965200.0         NaN  
       2013-01-04  594333600.0         NaN  
       2013-01-07  484156400.0         NaN  
       2013-01-08  458707200.0         NaN  
...                        ...         ...  
V      2024-01-24    5021700.0  121.734839  
       2024-01-25    6894900.0  105.245794  
       2024-01-26    8586500.0   21.563192  
       2024-01-29    6574500.0   88.247222  
       2024-01-30    6549000.0  161.879614  

[89475 rows x 7 columns]

In [141]:
dates = stocks_prices.index.get_level_values("Date")
bought = ("2014-01-01" <= dates) & (dates <= "2024-01-01") & (dates.dayofweek == 4) & (stocks_prices["cci"] > 200)

In [142]:
profit = ((stocks_prices["Adj Close"].shift(-5) / stocks_prices.loc[bought, "Adj Close"]) - 1) * 1000

In [143]:
profit.sum() * 1e-3

1.048264607975058

In [144]:
bought.sum()

460